In [1]:
import tensorflow as tf
import re
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive

drive.mount("/content/drive")

import numpy as np
import os
import time

file = open("/content/drive/My Drive/466 data/business.txt", "rb")
text = file.read().decode(encoding='utf-8')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

processed = re.sub(r'[^\x00-\x7f]', r'', text)

processed = processed.split()
#print(processed[:100])

maxlen = 5
sentences = []
for i in range(maxlen, len(processed)):
  seq = processed[i-maxlen:i]
  line = " ".join(seq)
  sentences.append(line)

sentences[1]





'profits at us media giant'

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

In [5]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]

vocab_size = len(tokenizer.word_index) + 1


y = to_categorical(y, num_classes = vocab_size)


In [6]:
model = Sequential()
model.add(Embedding(vocab_size, 256))
#model.add(GRU(1024, return_sequences=True))
model.add(GRU(1024))
model.add(Dense(1024, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))



In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         3420672   
_________________________________________________________________
gru (GRU)                    (None, 1024)              3938304   
_________________________________________________________________
dense (Dense)                (None, 1024)              1049600   
_________________________________________________________________
dense_1 (Dense)              (None, 13362)             13696050  
Total params: 22,104,626
Trainable params: 22,104,626
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
model.fit(X,y, batch_size=256, epochs=20)

Epoch 1/20
641/641 [==============================] - 69s 57ms/step - loss: 7.3573 - accuracy: 0.0653
Epoch 2/20
641/641 [==============================] - 35s 55ms/step - loss: 6.1200 - accuracy: 0.1333
Epoch 3/20
641/641 [==============================] - 36s 55ms/step - loss: 5.3915 - accuracy: 0.1746
Epoch 4/20
641/641 [==============================] - 35s 55ms/step - loss: 4.6918 - accuracy: 0.2135
Epoch 5/20
641/641 [==============================] - 35s 55ms/step - loss: 3.9062 - accuracy: 0.2723
Epoch 6/20
641/641 [==============================] - 35s 55ms/step - loss: 3.0685 - accuracy: 0.3667
Epoch 7/20
641/641 [==============================] - 35s 55ms/step - loss: 2.2536 - accuracy: 0.4987
Epoch 8/20
641/641 [==============================] - 35s 55ms/step - loss: 1.5715 - accuracy: 0.6301
Epoch 9/20
641/641 [==============================] - 35s 55ms/step - loss: 1.0309 - accuracy: 0.7487
Epoch 10/20
641/641 [==============================] - 35s 55ms/step - loss: 0.664

In [39]:
seed_text = sentences[100]
seed_text
seed_text = 'I am'

In [105]:
def generate_text(model, tokenizer, text_seq_len, seed_text, n_words):
  text = []
  word_distribution = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = 4, truncating='pre')

    temp = model.predict(encoded)

    y_predict = np.argmax(temp, axis=-1)
    #print(y_predict)

    word_list = []
    max_index = y_predict
    for x in range(4):
      temp[0][max_index] = -1
      max_index = np.argmax(temp, axis=-1)
      word_list.append(max_index)

    predicted_word = ''
    temp = model.predict(encoded)

    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)

    distribution = [(predicted_word, temp[0][y_predict][0])]
    for predicted in word_list:
      predicted_word = ''
      for word, index in tokenizer.word_index.items():
        if index == predicted:
          predicted_word = word
          break
      distribution.append((predicted_word, temp[0][predicted][0]))
    word_distribution.append(distribution)

  return ' '.join(text), word_distribution


In [106]:
text, distribution = generate_text(model, tokenizer, 4, seed_text, 10)

In [104]:
print(text)
for iteration in distribution:
  print("+++++++++++++++++")
  for word in iteration:
    print(word[0],":",word[1])

surprised at the weak fourth quarter number the main drag
+++++++++++++++++
surprised : [0.7230421]
confident : 0.13335705
not : 0.1239629
determined : 0.006177862
proud : 0.004528398
+++++++++++++++++
at : [0.999074]
by : 0.00040842313
that : 0.00037742427
against : 0.00011733388
about : 1.1494198e-05
+++++++++++++++++
the : [0.998672]
any : 0.0004050075
smith : 0.00020362882
about : 0.00017229751
a : 0.000169526
+++++++++++++++++
weak : [0.9739963]
early : 0.004896869
markets : 0.0041109696
2 : 0.0035028134
one : 0.0014541046
+++++++++++++++++
fourth : [0.9863625]
development : 0.007766148
dollar : 0.0045385677
economic : 0.00047902987
commission : 0.00040817904
+++++++++++++++++
quarter : [0.99981207]
time : 0.00018551444
meeting : 1.7083922e-06
fall : 5.997089e-07
have : 9.912486e-08
+++++++++++++++++
number : [0.9987973]
country : 0.00027925833
deficit : 0.00016901792
us : 0.00014253029
fall : 0.00012446639
+++++++++++++++++
the : [0.87981]
sales : 0.052858874
profits : 0.04813644